In [46]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals

In [47]:
import collections
import codecs
import re
import string
import stop_words
from nltk.stem.snowball import RussianStemmer
import pandas as pd

# Препроцессинг

In [48]:
data = pd.read_csv("laba1_1.csv", header=[0, 1], index_col=0, encoding='utf-8')

def str_to_set(st):
    st = st.split(u', ')
    return (st[0][1:-1], int(st[1]))

inver_index = collections.defaultdict(list)

with codecs.open("invert.index", "r", "utf-8") as f:
    for line in f:
        token = line.split(u' -> ')[0]
        keys = line.split(u' -> ')[1][2:-3].split(u'), (')
        for key in keys:
            inver_index[token].append(str_to_set(key))
            
EXCLUDE_SYMBOLS_STR = u''.join(['№', '«', 'ђ', '°', '±', '‚', 'ћ', '‰', '…', '»', 'ѓ', 'µ', '·', 'ґ', 'њ', 'ї', 'џ', 'є', '‹',
                                '‡', '†', '¶', 'ќ', '€', '“', 'ў', '§', '„', '”', '\ufeff', '’', 'љ', '›', '•', '—', '‘', 
                                '\x7f', '\xad', '¤', '\xa0'])
            
regex_punct = re.compile('[%s]' % re.escape(string.punctuation))
regex_dig = re.compile('[%s]' % re.escape(string.digits))
regex_symb = re.compile('[%s]' % re.escape(EXCLUDE_SYMBOLS_STR))
regex_struct = re.compile('[%s]' % string.printable + string.whitespace)

stemmer = RussianStemmer()

def unique_list(l):
    x = []
    for a in l:
        if a not in x:
            x.append(a)
    return x

def preprocessor(string):
    string = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', u'', string)
    string = re.sub(' +' , ' ', string)
    string = string.strip().strip('\t').replace('\n', u'')
    string = regex_punct.sub('', string)
    string = regex_dig.sub('', string)
    string = regex_symb.sub(' ', string)
    string = regex_struct.sub('', string)
    string = re.sub(' +' , ' ', string.strip())
    string = string.lower().split()
    tokens = []
    for word in string:
        if word not in stop_words.get_stop_words('ru'):
            tokens.append(word)
    return [stemmer.stem(word) for word in tokens]

# Поисковик 

In [92]:
def search(text, val):
    prep_text = preprocessor(text)
    indexes = set(inver_index[prep_text[0]])
    for token in prep_text[1:]:
        indexes.intersection_update(set(inver_index[token]))
    indexes = list(indexes)
    ind_per_text={}
    ind_per_text[" ".join(prep_text)]=indexes
    if len(indexes) < val:
        combinations = [prep_text]
        flag = False
        for item in combinations:
            if flag or len(item) <= 1:
                break
            for i, elem in enumerate(item):
                tmp = item[:]
                tmp.pop(i)
                combinations.append(tmp)
                tmp_indexes = set(inver_index[tmp[0]])
                for token in tmp[1:]:
                    tmp_indexes.intersection_update(set(inver_index[token]))
                ind_per_text[" ".join(tmp)]=tmp_indexes   
                indexes.extend(list(tmp_indexes))
                if len(indexes) >= val:
                    f = True
                    break
                    
        indexes = unique_list(indexes)[:val]
    else:
        indexes = indexes[:val]
    texts = []
    for index in indexes:
        texts.append({'Title': data[index[0]]['Title'][index[1]], 'Text': data[index[0]]['Text'][index[1]]})
    return texts

In [95]:
((texts, indexes), ind_per_text)= search("финал чм в россии", 15)

ValueError: too many values to unpack

In [96]:
for text, index in zip(texts, indexes):
    for key, item in ind_per_text.items():
        if index in item:
            text=key
            break
    print "Title:", item['Title']
    print text
    print "Text:", item['Text']
    print "****************************************************************************************************************************"

NameError: name 'indexes' is not defined